In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import geopy.distance
import geocoder
from tqdm import tqdm


In [40]:
df_train = pd.read_csv('train.csv', index_col=[0]).dropna()
df_test = pd.read_csv('test.csv', index_col=[0])

In [41]:
concat_df = pd.concat([df_train,df_test])
concat_df.head()

,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [42]:
concat_df[concat_df.lat.isna()]

,id,atm_group,address,address_rus,lat,long,target
1,2570.0,3185.5,UL. SERGEJA MAKEEVA MOSKVA,NaN,NaN,NaN,NaN
90,6662.0,5478.0,"D. 12, UL. AKADEMICHESKAY IRKUTSK G",NaN,NaN,NaN,NaN
99,484.0,1942.0,93 KRASNOARMEYSKAYA SAMARA,NaN,NaN,NaN,NaN
137,1469.0,1942.0,20 GUSINOBRODSKOE H/W NOVOSIBIRSK,NaN,NaN,NaN,NaN
146,3284.0,8083.0,"YA KOMSOMOLSK,19A Novo-Aleksandro",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2346,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN,NaN
2399,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN,NaN
2471,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN,NaN
2474,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN,NaN


In [32]:
import requests

def fetch_coordinates(apikey, address):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    response = requests.get(base_url, params={
        "geocode": address,
        "apikey": apikey,
        "format": "json",
    })
    response.raise_for_status()
    found_places = response.json()['response']['GeoObjectCollection']['featureMember']

    if not found_places:
        return None

    most_relevant = found_places[0]
    lon, lat = most_relevant['GeoObject']['Point']['pos'].split(" ")
    return most_relevant, lon, lat

# geocoder_data =fetch_coordinates('3c6b7b85-2712-4375-a6b8-bb47b4af147d','UL. SERGEJA MAKEEVA MOSKVA')

for index, row in concat_df[concat_df.lat.isna()].iterrows():
    idx = row['id']
    address = row['address']
    geocoder_data =fetch_coordinates('3c6b7b85-2712-4375-a6b8-bb47b4af147d',address)
    lon = geocoder_data[1]
    lat = geocoder_data[2]
    d.append({'index':idx,'long':lon,'lat':lat})
    
for coors in d:
    concat_df.loc[concat_df.id == coors['index'],'lat'] = float(coors['lat'])
    concat_df.loc[concat_df.id == coors['index'],'long'] = float(coors['long'])

In [33]:
geocoder_data

({'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'street',
     'text': 'Россия, Москва, улица Сергея Макеева',
     'kind': 'street',
     'Address': {'country_code': 'RU',
      'formatted': 'Россия, Москва, улица Сергея Макеева',
      'Components': [{'kind': 'country', 'name': 'Россия'},
       {'kind': 'province', 'name': 'Центральный федеральный округ'},
       {'kind': 'province', 'name': 'Москва'},
       {'kind': 'locality', 'name': 'Москва'},
       {'kind': 'street', 'name': 'улица Сергея Макеева'}]},
     'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва, улица Сергея Макеева',
       'CountryNameCode': 'RU',
       'CountryName': 'Россия',
       'AdministrativeArea': {'AdministrativeAreaName': 'Москва',
        'Locality': {'LocalityName': 'Москва',
         'Thoroughfare': {'ThoroughfareName': 'улица Сергея Макеева'}}}}}}},
   'name': 'улица Сергея Макеева',
   'description': 'Москва, Россия',
   'boundedBy': {'Envelope': {'lowerCorner':

In [35]:
geocoder_data[1:]

('37.551672', '55.763143')

In [38]:
geocoder_data[0]

{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'street',
    'text': 'Россия, Москва, улица Сергея Макеева',
    'kind': 'street',
    'Address': {'country_code': 'RU',
     'formatted': 'Россия, Москва, улица Сергея Макеева',
     'Components': [{'kind': 'country', 'name': 'Россия'},
      {'kind': 'province', 'name': 'Центральный федеральный округ'},
      {'kind': 'province', 'name': 'Москва'},
      {'kind': 'locality', 'name': 'Москва'},
      {'kind': 'street', 'name': 'улица Сергея Макеева'}]},
    'AddressDetails': {'Country': {'AddressLine': 'Россия, Москва, улица Сергея Макеева',
      'CountryNameCode': 'RU',
      'CountryName': 'Россия',
      'AdministrativeArea': {'AdministrativeAreaName': 'Москва',
       'Locality': {'LocalityName': 'Москва',
        'Thoroughfare': {'ThoroughfareName': 'улица Сергея Макеева'}}}}}}},
  'name': 'улица Сергея Макеева',
  'description': 'Москва, Россия',
  'boundedBy': {'Envelope': {'lowerCorner': '37.549965 55.7559

In [39]:
geocoder_data[0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['AdministrativeAreaName']

'Москва'

In [30]:
from decimal import Decimal

from yandex_geocoder import Client


client = Client("3c6b7b85-2712-4375-a6b8-bb47b4af147d")

coordinates = client.coordinates("UL. SERGEJA MAKEEVA MOSKVA")
print(coordinates)

(Decimal('37.551672'), Decimal('55.763143'))


In [50]:
d= []

for index, row in concat_df[concat_df.lat.isna()].iterrows():
    idx = row['id']
    address = row['address']
    geocoder_data =fetch_coordinates('3c6b7b85-2712-4375-a6b8-bb47b4af147d',address)
    lon = geocoder_data[1]
    lat = geocoder_data[2]
    d.append({'index':idx,'long':lon,'lat':lat})

In [52]:
d[0]

{'index': 2570.0, 'long': '37.551672', 'lat': '55.763143'}

In [57]:
for coors in d:
    concat_df.loc[concat_df.id == coors['index'],'lat'] = float(coors['lat'])
    concat_df.loc[concat_df.id == coors['index'],'long'] = float(coors['long'])

In [58]:
concat_df[concat_df.lat.isna()]

,id,atm_group,address,address_rus,lat,long,target


In [61]:
concat_df.to_csv('concat_df.csv')

In [10]:
# Список OSM тэгов
tag_list = [
    # {'subway': ['yes']},
    {'public_transport': ['stop_position']},
    {'government': ['administrative']},
    {'amenity': ['college', 'university']},
    {'amenity': ['school']},
    {'amenity': ['kindergarten', 'childcare']},
    {'office': ['company']},
    {'amenity': ['cafe', 'fast_food', 'bar', 'pub', 'canteen']}
]

# Human-readable имена тегов 
tag_names = [
    # 'subway',
    'bus_stop',
    'gov_building',
    'college',
    'school',
    'kgarden',
    'office',
    'food'
]
        
    # (
    #     node["subway"~"yes"](around:{radius},{lat}, {lon});
    #     node["public_transport"~"stop_position"](around:{radius},{lat}, {lon});
    #     node["government"~"administrative"](around:{radius},{lat}, {lon});
    #     node["amenity"~"college|university"](around:{radius},{lat}, {lon});
    #     node["amenity"~"school"](around:{radius},{lat}, {lon});
    #     node["amenity"~"kindergarten|childcare"](around:{radius},{lat}, {lon});
    #     node["office"~"company"](around:{radius},{lat}, {lon});
    #     node["amenity"~"cafe|fast_food|bar|pub|canteen"](around:{radius},{lat}, {lon});
    # );
    # out body;

In [20]:
import overpy
import time
api = overpy.Overpass()

def query(api,radius,lat,lon,tag_primary,tag_secondary):
    try:
        result = api.query("""
            node(around:{radius},{lat},{lon})["{tag_primary}"~"{tag_secondary}"];
            out body;
            """.format(radius=radius,lat=lat,lon=lon,tag_primary=tag_primary,tag_secondary=tag_secondary))
    except Exception as e:
        time.sleep(30)
        result = api.query("""
            node(around:{radius},{lat},{lon})["{tag_primary}"~"{tag_secondary}"];
            out body;
            """.format(radius=radius,lat=lat,lon=lon,tag_primary=tag_primary,tag_secondary=tag_secondary))
    return result

res = query(api,10000,55.805827,37.515146,'amenity','college|university')

def process_query(api,lat,lon,tag_primary,tag_secondary):
    res = query(api,10000,lat,lon,tag_primary,tag_secondary)
    distances = []
    for node in res.nodes:
        coords_1 = (lat,lon)
        coords_2 = (node.lat,node.lon)
        distances.append(geopy.distance.geodesic(coords_1, coords_2).m)
    series = pd.Series(distances)
    if len(distances) == 0:
        return 10000,0,0
    min_dist = series.min()
    count_250m = (series <= 250).sum()
    count_500m = (series <= 500).sum()
    return(min_dist,count_250m,count_500m)

In [21]:
process_query(api,55.805827,37.515146,'cafe','cafe|fast_food|bar|pub|canteen')

(610.1036698194454, 0, 0)

In [ ]:
for i in range(len(tag_names)):
    column_name = tag_names[i]
    primary_tag = list(tag_list[i].keys())[0]
    secondary_tag = '|'.join(tag_list[i][primary_tag])
    print('current tag: ',column_name,' tags: ',primary_tag,secondary_tag)
    min_dists,count_250ms,count_500ms = [],[],[]
    for index, row in tqdm(concat_df.iterrows(), total=concat_df.shape[0]):
        min_dist,count_250m,count_500m = process_query(api,row['lat'],row['long'],primary_tag,secondary_tag)
        min_dists.append(min_dist)
        count_250ms.append(count_250m)
        count_500ms.append(count_500m)
    concat_df['min_'+column_name] = pd.Series(min_dists)
    concat_df['250m_'+column_name] = pd.Series(count_250ms)
    concat_df['500m_'+column_name] = pd.Series(count_500ms)


current tag:  bus_stop  tags:  public_transport stop_position


  0%|                                       | 12/8461 [01:21<9:04:12,  3.86s/it]/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_42993/3242676869.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series(distances)
 44%|███████████████▍                   | 3734/8461 [1:35:32<1:24:17,  1.07s/it]

In [24]:
concat_df.to_csv('concat_df.csv')

In [23]:
concat_df

,id,atm_group,address,address_rus,lat,long,target,min_subway,250m_subway,500m_subway
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500,10000.000000,0,0
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710,10000.000000,0,0
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540,10000.000000,0,0
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035,59.371791,3,3
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277,10000.000000,0,0
...,...,...,...,...,...,...,...,...,...,...
2499,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN,7380.565971,0,0
2500,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN,10000.000000,0,0
2501,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN,10000.000000,0,0
2502,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN,10000.000000,0,0
